In [2]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [2]:
# Resets the default global graphs.
tf.compat.v1.get_default_graph()
tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()   # by defaulut it is enabled

In [3]:
# Creating constant tensors
a=tf.constant(value=5,name='constant_a')    # we have a tensor 'a' with value 5 and name as 'constant_a'
b=tf.constant(100,name='constant_b')
c=tf.constant(16,name='constant_c')
d=tf.constant(8,name='constant_d')

Link for referring to different operators in tf -<br>
https://www.tensorflow.org/api_docs/python/tf

In [4]:
# Computation(Operator) using Constant (Nodes to the graph)
mul=tf.multiply(a,b,name='mul')           # Operator 'multiply' with name 'mul'
mul2=tf.multiply(c,d,name='mul2')
#div=tf.cast(div,'int32')
addn=tf.add_n([mul, mul2],name='addn')

In [5]:
# info about tensor
# Since, we have disbaled eager_execution
# the computaton won't be performed  as we are just building the graph and it's not executed it.
print(addn)    

Tensor("addn:0", shape=(), dtype=int32)


### Session :-
Encapsulates the environment where ```Operation``` objects are executed and ```Tensor``` objects are evaluated

In [6]:
sess=tf.compat.v1.Session()
print(sess.run(addn))   # computes the result and envokes the computation graph
print(sess.run(mul))

628
500


1. All the api that are used above are low level api's which we use to build our graph in Production.
2. `tensorboard` helps to visualise our graph and can be used for debugging.
3. To do that we need to instrument our code by writing summary and events in our code

In [7]:
# Creates an event file and constantly updates that file with events and summaries
writer=tf.compat.v1.summary.FileWriter('./constants',sess.graph)   # sess.graph holds the current state of our computation graph

writer.close()  # frees up an resources that it is holding.

RUN THE TERMINAL COMMAND TO RUN TENSORBOARD :-<br>
`tensorboard --logdir='./constants'/`

## Variables

In [8]:
tf.compat.v1.reset_default_graph()

In [9]:
# declaring varibales in tf
m=tf.compat.v1.Variable([2.5,4.0],tf.float32,name='variable_m')
c=tf.compat.v1.Variable([5.0,10.0],tf.float32,name='variable_c')
m

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<tf.Variable 'variable_m:0' shape=(2,) dtype=float32>

In [10]:
# from above varibale name we are trying to fit in the line with y=mx+c
# Placeholder or empty container for tensor values which will be fed in during the execution
x=tf.compat.v1.placeholder(tf.float32,shape=[2],name='placeholder_x')
x

<tf.Tensor 'placeholder_x:0' shape=(2,) dtype=float32>

In [11]:
# equation of the line
y=m*x+c
y

<tf.Tensor 'add:0' shape=(2,) dtype=float32>

In [12]:
# Before using any variables we need to explicitly initialise it
# Operator needs to be called to initialise all the variables in th graph
init=tf.compat.v1.global_variables_initializer()

In [13]:
with tf.compat.v1.Session() as sess:
    sess.run(init)
    y_output=sess.run(y,feed_dict={x:[20.0,50.0]})
    print("Final result:mx + c",y_output)
    writer=tf.compat.v1.summary.FileWriter('./variables_and_placeholders',sess.graph)
    writer.close()

Final result:mx + c [ 55. 210.]
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
tf.float32 has type DType, but expected one of: int, long, bool
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
tf.float32 has type DType, but expected one of: int, long, bool


In [14]:
y_output

array([ 55., 210.], dtype=float32)

### Initialising only a particular variable in place of all the variables

In [17]:
s=m*x
init=tf.compat.v1.variables_initializer([m])   # initialise only variable m
with tf.compat.v1.Session() as sess:
    sess.run(init)
    print("Result m*n is :{}".format(sess.run(s,feed_dict={x:[20,200]})))
  # print("Result y :{}".format(sess.run(y,feed_dict={x:[20,200]})))   # this will produce an error as c is not init

Result m*n is :[ 50. 800.]


### Updating a Variable within the Session

In [19]:
doubler=tf.compat.v2.Variable(1)      # Doubles the value in each iteration.
incrementor=tf.compat.v2.Variable(1)  # Increments the value by one.

init=tf.compat.v1.global_variables_initializer()

In [20]:
result=doubler.assign(tf.add(doubler,doubler))


with tf.compat.v1.Session() as sess:
    sess.run(init)
    
    for i in range(10):
        print("Value of Doubler :{}".format(sess.run(result)))
        
    for i in range(10):
        print("Value of incrementor :{}".format(sess.run(incrementor.assign_add(1))))

Value of Doubler :2
Value of Doubler :4
Value of Doubler :8
Value of Doubler :16
Value of Doubler :32
Value of Doubler :64
Value of Doubler :128
Value of Doubler :256
Value of Doubler :512
Value of Doubler :1024
Value of incrementor :2
Value of incrementor :3
Value of incrementor :4
Value of incrementor :5
Value of incrementor :6
Value of incrementor :7
Value of incrementor :8
Value of incrementor :9
Value of incrementor :10
Value of incrementor :11


Thus from the above code we can understand that the variables are trainable parameters.

### Understanding fetch_dict with linear equations

In [21]:
# Resetting the default graph
tf.compat.v1.reset_default_graph()

In [22]:
# Declaring 1D tensor vectors
a=tf.constant([1,10],name='constant_a')
b=tf.constant([10,100],name='constant_b')
c=tf.constant([100,1000],name='constant_c')

In [28]:
# Declaring place holders i.e. weights for respective constants
x_1=tf.compat.v1.placeholder(tf.int32,name='x_1')
x_2=tf.compat.v1.placeholder(tf.int32,name='x_2')
x_3=tf.compat.v1.placeholder(tf.int32,name='x_3')

#multiplying each placeholder with corresponding constants
ax_1=tf.multiply(a,x_1,name='ax_1')
bx_2=tf.multiply(b,x_2,name='bx_2')
cx_3=tf.multiply(c,x_3,name='cx_3')

# Calculating y_sub
with tf.name_scope('subtract'):
    y_sub=tf.subtract(bx_2,ax_1,name='y_sub')

# Calculating y_add
with tf.name_scope('Add'):
    y_add=ax_1+bx_2+cx_3
with tf.name_scope('Final'):
    y_final=y_add * y_sub

In [29]:
with tf.compat.v1.Session() as sess:
    print("Value of ax_1: {}".format(sess.run(fetches=ax_1,feed_dict={x_1:[4,44]})))
    
    print("Value of y_sub: {}".format(sess.run(fetches=y_sub,feed_dict={x_1:[4,44],x_2:[7,77]})))
    
    print("Value of y_final: {}".format(sess.run(y_final,feed_dict={x_1:[4,44],x_2:[7,77],x_3:[2,22]})))
    
    writer=tf.compat.v1.summary.FileWriter('./fetches_feeddict',sess.graph)
    
    writer.close()

Value of ax_1: [  4 440]
Value of y_sub: [  66 7260]
Value of y_final: [    18084 218816400]


When visualising the graph we can see a graph without any order. Hence, we can group certain operations together<br>
by using something called as the name_scope().

### Understanding Eager Execution 
Till now we have executed the code in static mode where Building a graph and Executing it are in two different phases. Well now by enhancement in modules we can combine these two discrete steps into one by the added functionality called as `Eager Execution`. By default in tf v2.1.0 it is automatically enabled but to start it again we can use the following command.To do this restart the kernel.

In [3]:
import numpy as np

In [4]:
# We'll start by creating a Multidimensional numpy array
a=np.array([[10,11],[12,13]])
b=np.ones([2,2])

In [5]:
sum=tf.add(a,b)
sum    # Hence we can see that the tensors are already created and Executed in a single go.

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[11, 12],
       [13, 14]])>

In [6]:
sqr=tf.square(a)
sqr

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[100, 121],
       [144, 169]])>

NOTE :-<BR>
    For Production task in the industry people prefer to use the static mode as it can use the ability of the  CPUs and GPUs to perform task parallely.